## EXTRACCIÓN DE CARACTERÍSTICAS
Este notebook extrae glosarios para las distintas categorías de texto utilizando la librería gensim.

In [40]:
!pip3 install gensim 

In [54]:
import pandas as pd
import os
import gensim
from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

Cargamos nuestro dataframe de noticias dedicado al entrenamiento

In [42]:
noticias_dataframe = pd.read_csv('../Datos/noticias_train.csv')
noticias_dataframe_t = pd.read_csv('../Datos/noticias_test.csv')

Separamos nuestras noticias en diferentes cojuntos de datos por categoría de noticia

In [43]:
deportes_dataframe = noticias_dataframe[noticias_dataframe["category"]=="sports"]
deportes_dataframe.info()

print("-"*60)

deportes_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="sports"]
deportes_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 

In [44]:
salud_dataframe = noticias_dataframe[noticias_dataframe["category"]=="health"]
salud_dataframe.info()

print("-"*60)

salud_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="health"]
salud_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 30 to 59
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 30 to 59
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object

In [45]:
ciencia_dataframe = noticias_dataframe[noticias_dataframe["category"]=="science"]
ciencia_dataframe.info()

print("-"*60)

ciencia_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="science"]
ciencia_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 60 to 89
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 60 to 89
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object

In [46]:
politica_dataframe = noticias_dataframe[noticias_dataframe["category"]=="politics"]
politica_dataframe.info()

print("-"*60)

politica_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="politics"]
politica_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 90 to 119
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 90 to 119
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     obje

### Extracción de glosario mediante TFIDF


La forma más sencilla para obtener nuestro glosario es mediante el esquema *tf-idf*

In [47]:
def extrae_glosario_tf_idf(categoria_dataframe, size):
    """ 
    Funcion para extraer el glosario

    - categoria_dataframe: cojunto de datos de una categoriad de noticias
    - size: tokens del glosario a mostrar
    """
    # Pre-procesamos el corpus de cada noticia
    doc_tokens = [simple_preprocess(corpus) for corpus in categoria_dataframe["corpus"]]

    # Hacemos un diccionario. Mapeo de palabras e identificadores de palabra
    dictionary = corpora.Dictionary(doc_tokens)

    # Convertimos el diccionario al formato BoW
    bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokens]

    # https://radimrehurek.com/gensim/models/tfidfmodel.html
    # Implementamos un modelo tf-idf
    tfidf = models.TfidfModel(bow_corpus, smartirs="lfc")
    bow_tfidf = tfidf[bow_corpus]

    # Por cada documento, sacamos un glosario
    tfidf_dic = {dictionary.get(id): value for doc in bow_tfidf for id, value in doc}
    tfidf_list = [k for k, v in sorted(tfidf_dic.items(), key=lambda item: item[1], reverse = True)]

    return tfidf_list[:size]
    

In [48]:
glosarios = {'deportes': [],
             'salud': [],
             'ciencia': [],
             'salud': []}

glosarios_t = {'deportes': [],
            'salud': [],
            'ciencia': [],
            'salud': []}

In [49]:
glosarios['deportes'] = extrae_glosario_tf_idf(deportes_dataframe, 50)
print(glosarios['deportes'])

print("-"*60)

glosarios_t['deportes'] = extrae_glosario_tf_idf(deportes_dataframe_t, 50)
print(glosarios_t['deportes'])


['falso', 'jamas', 'smash', 'mbappe', 'gigante', 'reserva', 'nets', 'carpena', 'djokovic', 'mclaren', 'exencion', 'butler', 'cuento', 'formato', 'campazzo', 'tenerife', 'domenicali', 'booker', 'tatum', 'madrid', 'real', 'premio', 'juventus', 'pagar', 'ronaldo', 'magnussen', 'steiner', 'gavi', 'porra', 'indycar', 'palou', 'gonzalez', 'gonzalo', 'rotacion', 'florentino', 'heredero', 'verdasco', 'enrique', 'luis', 'cristiano', 'cd', 'marbella', 'residencia', 'serbia', 'serbio', 'rosa', 'collins', 'jimmy', 'arabia', 'club']
------------------------------------------------------------
['seguidor', 'carlos', 'mans', 'warren', 'doncic', 'suarez', 'juancho', 'boston', 'estabilidad', 'horford', 'hernanga', 'mez', 'ferrero', 'enrique', 'grada', 'mirar', 'marko', 'wiggins', 'stakhovsky', 'raptors', 'booker', 'resistencia', 'pts', 'reb', 'colombia', 'garden', 'brooklyn', 'cancha', 'lebron', 'gasto', 'tristeza', 'celtics', 'gimenez', 'sancionar', 'siebert', 'ktm', 'moto', 'dolares', 'rez', 'golden'

In [50]:
glosarios['salud'] = extrae_glosario_tf_idf(salud_dataframe, 30)
print(glosarios['salud'])

print("-"*60)

glosarios_t['salud'] = extrae_glosario_tf_idf(salud_dataframe_t, 30)
print(glosarios_t['salud'])

['arterial', 'dash', 'anemia', 'folcodina', 'cafa', 'genital', 'vrs', 'unicef', 'lobo', 'verruga', 'aceite', 'mascarilla', 'pulmonar', 'presion', 'cannabis', 'pet', 'epilepsia', 'zumo', 'quiraorgica', 'afiliado', 'eps', 'ips', 'tc', 'creatividad', 'condiloma', 'marfan', 'manada', 'parasito', 'lavar', 'coosalud']
------------------------------------------------------------
['strep', 'estigma', 'secuela', 'automatico', 'orina', 'antidepresivo', 'aprovechar', 'nuez', 'entrenar', 'bacteria', 'fruto', 'seco', 'cabello', 'atras', 'sida', 'vih', 'esconder', 'temporada', 'receta', 'congelado', 'alga', 'wakame', 'rehabilitacia', 'adolescente', 'gotlib', 'depresion', 'bronquiolitis', 'cuadro', 'niaos', 'morder']


In [51]:
glosarios['politica'] = extrae_glosario_tf_idf(politica_dataframe, 30)
print(glosarios['politica'])

print("-"*60)

glosarios_t['politica'] = extrae_glosario_tf_idf(politica_dataframe_t, 30)
print(glosarios_t['politica'])

['rosell', 'vuelo', 'sentir', 'reyes', 'rufian', 'unilateral', 'multilateral', 'ja', 'sinema', 'benito', 'juarez', 'turismo', 'ucraniano', 'nicolas', 'negativo', 'autoritariaa', 'ruido', 'nero', 'registral', 'demarcacia', 'ex', 'consulta', 'patrimonio', 'perao', 'juventud', 'trias', 'dema', 'coraza', 'ciudadano', 'anunciara']
------------------------------------------------------------
['silva', 'refera', 'ilacito', 'rodraguez', 'torra', 'insulto', 'urgente', 'esparza', 'mocia', 'animal', 'castillo', 'almeida', 'smith', 'upn', 'cheque', 'ayres', 'concejala', 'mexicano', 'catalunya', 'isabel', 'ndum', 'nduma', 'ahorro', 'nombramiento', 'obrador', 'adjudicacia', 'gandia', 'procesado', 'navarra', 'auxiliar']


In [52]:
glosarios['ciencia'] = extrae_glosario_tf_idf(ciencia_dataframe, 30)
print(glosarios['ciencia'])

print("-"*60)

glosarios_t['ciencia'] = extrae_glosario_tf_idf(ciencia_dataframe_t, 30)
print(glosarios_t['ciencia'])

['vacao', 'llama', 'captura', 'cola', 'honor', 'gemanidas', 'banyoles', 'neandertal', 'agujero', 'latigo', 'supersa', 'perfecto', 'regalo', 'bankman', 'fried', 'navidad', 'estampido', 'ftx', 'york', 'menta', 'supermasivo', 'barrera', 'segundo', 'ingeniero', 'magic', 'reencuentro', 'leo', 'nif', 'cernan', 'fauci']
------------------------------------------------------------
['sal', 'cuantico', 'idioma', 'fraa', 'vacuna', 'isidro', 'santo', 'llnl', 'agujero', 'sonido', 'antageno', 'subtipo', 'quipus', 'congelacia', 'ska', 'ofensivo', 'palabrota', 'canaria', 'relatividad', 'estruendo', 'programador', 'telescopio', 'congelar', 'gripe', 'alineacia', 'almanaque', 'escarcha', 'helada', 'sevilla', 'alphacode']


Guardamos los glosarios en .txt

In [61]:
if os.path.exists("../Datos/Glosarios") == False:
    os.mkdir("../Datos/Glosarios")
    
if os.path.exists("../Datos/Glosarios/train") == False:
    os.mkdir("../Datos/Glosarios/train")

if os.path.exists("../Datos/Glosarios/test") == False:
    os.mkdir("../Datos/Glosarios/test")


In [62]:
inicial = "../Datos/Glosarios/train/glosario"

for g in glosarios.items():
    tipo = g[0]
    fname = f"{inicial}_{tipo}.txt"
    with open(fname, 'w') as f:
        for termino in g[1]:
            f.write(termino)
            f.write('\n')

In [63]:
inicial = "../Datos/Glosarios/test/glosario"
for g in glosarios_t.items():
    tipo = g[0]
    fname = f"{inicial}_{tipo}.txt"
    with open(fname, 'w') as f:
        for termino in g[1]:
            f.write(termino)
            f.write('\n')
        